# Cleaning and Preprocessing 
### Summary of Activity

- **Load and Clean Data**:  
  We load data for benchmark index prices and constituent prices from CSV files, ensuring numeric columns are properly formatted and the `Date` column is converted to datetime.
  
- **Data Cleaning**:  
  This data originates from the Bloomberg terminal and uses BBG tickers. We remove unwanted suffixes (e.g., " Equity") from column names for clarity  
  We reshape the data using `melt` to create a tidy format with columns for `Date`, `Ticker`, and `Price`.
  
- **Date Range Filtering**:  
  We filter the data to include only rows within a specified date range (e.g.`1 Jan 2015` to `30 Sep 2024`).

- **Return Calculation**:  
  We extend the dataframe by calculating returns for various periods (1 business day, 1 week, 1 month, 1 quarter, and 1 year), handling missing values using a lookback window of 5 business days.
  
- **Data Merging**:  
  We merge the benchmark and constituent data to compute active returns by subtracting benchmark returns from constituent returns.

- **Validation**:  
  We verify that the active returns sum correctly by checking if the `active_returns` plus benchmark returns equals the total return.
  

- **Final Output**:  
  We save the cleaned, processed, and thresholded active returns to CSV files for further analysis.


In [ ]:
import numpy as np
import pandas as pd
import time
from pandas.tseries.offsets import BDay, Week, MonthEnd, QuarterEnd, YearEnd
from functions.constants import BM_NAME, STARTDATE, ENDDATE, N_THRESHOLD_BPS, DATA_DIR
from functions.helper_fns import remove_BBG_suffixes, melt_data

In [ ]:
# Load data
bm_index_prices_df_path = DATA_DIR + BM_NAME + "_BM_prices.csv"
bm_holdings_prices_df_path = DATA_DIR + BM_NAME + "_constituents_prices.csv"

def load_bm_index_df(path):
    df = pd.read_csv(path, skiprows=3)
    df = df.rename(columns={df.columns[0]: "Date"})
    df = df[2:]
    #ensure all cols except Date are numeric. Coerce nan if not
    for col in df.columns[1:]:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df["Date"] = pd.to_datetime(df["Date"])
    return df

bm_index_prices = load_bm_index_df(bm_index_prices_df_path)
bm_holdings_prices = load_bm_index_df(bm_holdings_prices_df_path)
print("Raw data: BM prices")
print(bm_index_prices.head())
print("Raw data: BM holdings prices")
print(bm_holdings_prices.head())

In [ ]:
def filter_date_range(df, start_date=STARTDATE, end_date=ENDDATE):
    """
    Filter the dataframe to keep only the rows within the specified date range
    """
    df = df[(df["Date"] >= start_date) & (df["Date"] <= end_date)]
    return df

def extend_with_returns(df, periods=["1b"], lookback_window='5b'):
    '''
    Extend the input dataframe with additional return columns for specified periods.
    Handles holidays and missing data by looking back up to a defined number of business days.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        The input dataframe containing at least 'Date', 'Ticker', and 'Price' columns.
    periods : list, optional, default=["1b"]
        A list of periods for which returns are calculated. Supported values include:
        '1b' (1 business day), '1w' (1 week), '1m' (1 month), '1q' (1 quarter), '1y' (1 year), 
        NOT YET SUPPORTED: 'mtd' (month-to-date), 'qtd' (quarter-to-date), 'ytd' (year-to-date).
    lookback_window : str, optional, default='5b'
        The lookback window for calculating returns when frequency-based returns are unavailable. 
        The default is 5 business days.

    Returns:
    --------
    pandas.DataFrame
        A new dataframe with additional columns for each specified return period (e.g., 'returns_1b', 'returns_mtd').
    '''
    # Ensure Date column is datetime and set it as index
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index('Date')
    
    # Sort the dataframe by Date and Ticker
    df = df.sort_index().sort_values('Ticker')
    
    # Define offset mappings for supported periods
    offset_map = {
        '1b': BDay(1),
        '1w': Week(1),
        '1m': MonthEnd(1),
        '1q': QuarterEnd(1),
        '1y': YearEnd(1)
    }
    # Extract number of business days from the lookback_window
    lookback_periods = int(lookback_window[:-1])  # Remove the 'b' and convert to int
    lookback_offset = BDay(lookback_periods)

    for period in periods:
        t = time.time()
        if period in offset_map:
            offset = offset_map[period]
            # Calculate returns based on standard periods
            df[f'returns_{period}'] = df.groupby('Ticker')['Price'].apply(
                lambda x: x.pct_change(freq=offset).fillna(x.pct_change(freq=lookback_offset))
            )
        else:
            # For non-standard periods, apply lookback window logic
            df[f'returns_{period}'] = df.groupby('Ticker')['Price'].apply(
                lambda x: x.pct_change(periods=lookback_periods).fillna(x.pct_change(periods=lookback_periods))
            )
        # Handling missing longer returns with a rolling window (e.g., '1w', '1m')
        if period in ['1w', '1m', '1q', '1y']:
            df[f'returns_{period}'] = df.groupby('Ticker')[f'returns_{period}'].fillna(method='ffill')
        print("Calculating returns for period:", period, "took", time.time() - t, "seconds")
    
    df = df.reset_index()
    return df

def preprocess_and_clean_data(df):
    t = time.time()
    df = remove_BBG_suffixes(df)
    print(f"Time taken to remove BBG suffixes: {time.time() - t:.6f} seconds")
    t = time.time()
    df = melt_data(df)
    print(f"Time taken to melt data: {time.time() - t:.6f} seconds")
    t = time.time()
    df = extend_with_returns(df, periods=["1b","1w", "1m", "1q", "1y"])
    print(f"Time taken to extend with returns: {time.time() - t:.6f} seconds")
    t = time.time()
    df = filter_date_range(df)
    print(f"Time taken to filter date range: {time.time() - t:.6f} seconds")
    return df

bm_index_prices_preproc = preprocess_and_clean_data(bm_index_prices)
print("Preprocessed data: BM prices")
print(bm_index_prices_preproc.head())
bm_holdings_prices_preproc = preprocess_and_clean_data(bm_holdings_prices)
print("Preprocessed data: BM holdings prices")
print(bm_holdings_prices_preproc.head())

In [ ]:
#print percentage of missing in each column
print("After having handled missing values due to holidays and weekends:\n")
print("Fraction of missing values in BM index prices:")
print(bm_index_prices_preproc.isnull().mean())
print("Fraction of missing values in BM holdings prices:")
print(bm_holdings_prices_preproc.isnull().mean())

In [ ]:
def get_active_returns(bm_index_prices, bm_holdings_prices):
    #establish a safe local scope
    bm_index_prices = bm_index_prices.copy()
    bm_holdings_prices = bm_holdings_prices.copy()

    holdings_cols = bm_holdings_prices.columns
    inferred_periods = [col for col in holdings_cols if 'returns' in col]
    inferred_periods = [col.split('_')[1] for col in inferred_periods]
    #rename bm_index_prices columns to have "bm_returns_" prefix. 
    bm_index_prices.columns = [col if col == "Date" else "bm_" + col for col in bm_index_prices.columns]
    # print(bm_index_prices.head())
    #merge the two dataframes on date left
    merged = pd.merge(bm_holdings_prices, bm_index_prices, on="Date", how="left")
    #calculate active returns
    for period in inferred_periods:
        merged["active_returns_" + period] = merged["returns_" + period] - merged["bm_returns_" + period]
    # print(merged.head())
    
    #TEST TO ENSURE THAT THE ACTIVE RETURNS ARE CORRECT AND SUMS TO WHEN ADDING BM RETURNS AND SUBTRACTING BM RETURNS
    merged_for_testing = merged.copy()
    merged_for_testing["TEST_COL"] = merged_for_testing["active_returns_1b"] + merged_for_testing["bm_returns_1b"] - merged_for_testing["returns_1b"]
    merged_for_testing["TEST_COL"] = merged_for_testing["TEST_COL"].fillna(0)  # Replace NaNs with zeros for comparison
    assert np.allclose(merged_for_testing["TEST_COL"], 0), "Active returns are not correctly calculated"

    #keep only (TICKER, Date, active_returns_*) columns
    active_returns = merged[["Ticker", "Date"] + [col for col in merged.columns if "active_returns" in col]]
    return active_returns

active_returns = get_active_returns(bm_index_prices_preproc, bm_holdings_prices_preproc)
active_returns

In [ ]:
active_returns_thresholded = active_returns.copy()
active_returns_thresholded = active_returns_thresholded.set_index(["Ticker", "Date"])
active_returns_thresholded = active_returns_thresholded > (N_THRESHOLD_BPS / 10000)
active_returns_thresholded = active_returns_thresholded.astype(float)
active_returns_thresholded = active_returns_thresholded.reset_index()
active_returns_thresholded

In [ ]:
active_returns_path = DATA_DIR + BM_NAME + "_active_returns.csv"
active_returns.to_csv(active_returns_path)
print("Active returns saved to " + active_returns_path)
active_returns_thresholded_path = DATA_DIR + BM_NAME + "_active_returns_thresholded_" + str(N_THRESHOLD_BPS) + "bps.csv"
active_returns_thresholded.to_csv(active_returns_thresholded_path)
print("Active returns thresholded saved to " + active_returns_thresholded_path)